In [1]:
from import_datasets import *
from utils_c import *

In [2]:
def getRecommendationsDf(recommendations):
    recom_df=[]
    for s in recommendations:
        itemset=[]
        for k, v in list(s.filters.items()):
            values = ''
            if k in encoders:
                le = encoders[k]
                for v_ in v:
                    values += '%s'%(le.inverse_transform(v_)[0])
            else:
                for v_ in v:
                    values += '%s'%(v_[0])
            itemset.append(f"{k}={values}")

        recom_df.append([frozenset(itemset), s.effect_size, s.metric, s.size])
    recom_df=pd.DataFrame(recom_df, columns=["itemset", "effect_size", "metric", "size"])
    return recom_df

# Dataset

In [3]:
import numpy as np
import pandas as pd
dataset_name="artificial_10"
features_3=["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]

data=np.concatenate([np.full((25000, 10), 0), np.full((25000, 10), 1)])
np.random.seed(7)
for i in range(data.shape[1]):
    np.random.shuffle(data[:,i])
df_artificial_3=pd.DataFrame(data=data, columns=features_3)
df_artificial_3["class"]=0
indexes=df_artificial_3.loc[((df_artificial_3["a"]==df_artificial_3["b"])& (df_artificial_3["a"]==df_artificial_3["c"]))].index
df_artificial_3.loc[indexes, "class"]=1
df_artificial_3["class"].value_counts()



type_cl="RF"
labelEncoding=True
X_FP, y_FP, y_predicted, y_predict_prob, encoders, indexes_FP, clf_3=train_predict(df_artificial_3, type_cl=type_cl, labelEncoding=labelEncoding, validation="all", retClf=True)
attributes=df_artificial_3.columns.drop("class")
#X_discretized=discretize(df_artificial_3, attributes=attributes, indexes_FP=indexes_FP)
X_one_hot_FP=oneHotEncoding(X_FP)

print("X")
display(X_FP.head())
print("y")
display(y_FP.head())
print("X_one_hot_FP")
display(X_one_hot_FP.head())

### Alteration

df_artif_altered_3=X_FP.copy()
df_artif_altered_3["class"]=y_FP["class"].copy()


indexes1=list(df_artif_altered_3.loc[\
    (df_artif_altered_3["a"]==df_artif_altered_3["b"]) &\
    (df_artif_altered_3["a"]==df_artif_altered_3["c"])].index)
import random
random.seed(7) 
random.shuffle(indexes1)
indexes=list(indexes1)[0:int(len(indexes1)/2)]
df_artif_altered_3.loc[indexes, "class"]=0

attributes_3=features_3
attributes_artif_one_hot_3=X_one_hot_FP.columns
if labelEncoding:
    X_altered_3=df_artif_altered_3[attributes_3]
    y_altered_3=df_artif_altered_3[["class"]]
    X_altered_one_hot_encoding_3=pd.get_dummies(df_artif_altered_3, prefix_sep="=", columns=attributes_3)[attributes_artif_one_hot_3]
else:
    X_altered_3=pd.get_dummies(df_artif_altered_3, prefix_sep="=", columns=attributes_3)[attributes_artif_one_hot_3]
    y_altered_3=df_artif_altered_3[["class"]]

y_altered_predicted_3=clf_3.predict(X_altered_3)
y_altered_predicted_prob_3=clf_3.predict_proba(X_altered_3)

X


,a,b,c,d,e,f,g,h,i,j
0,1,1,1,1,1,1,0,1,1,0
1,1,1,0,1,1,0,0,1,0,1
2,1,1,0,1,0,0,1,0,1,1
3,0,1,0,1,0,1,1,1,1,0
4,0,0,1,1,0,1,1,0,0,1


y


,class
0,1
1,0
2,0
3,0
4,0


X_one_hot_FP


,a=0,a=1,b=0,b=1,c=0,c=1,d=0,d=1,e=0,e=1,f=0,f=1,g=0,g=1,h=0,h=1,i=0,i=1,j=0,j=1
0,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0
1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1
2,0,1,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1
3,1,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,0
4,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,1,0,0,1


In [4]:
y_FP=y_altered_3
y_predict_prob=y_altered_predicted_prob_3
y_predicted=y_altered_predicted_3

In [5]:
from collections import Counter
print("True Class: ", Counter(y_FP["class"]))
print("Predicted class:", Counter(y_predicted))

from sklearn.metrics import accuracy_score
print("Accuracy: ", round(accuracy_score(y_FP, y_predicted),4))

True Class:  Counter({0: 43694, 1: 6306})
Predicted class: Counter({0: 37388, 1: 12612})
Accuracy:  0.8739


In [6]:
from statistics import mean
import time
n_times=10
times_sf={}

# DivExplorer

In [7]:
from utils_c import *
from utils_metrics_FPx import *
from FP_DivergenceExplorer import *
from item_score_FP import *
ITEMSETS=["support", "itemsets", "tn", "tp", "fn", "fp"]
DIVERGENCE_METRICS=["d_fpr", "d_fnr", "d_accuracy"] 
CLASSIFICATION_METRICS=["fpr", "fnr", "accuracy"]
T_VALUES=["t_value_fp", "t_value_fn", "t_value_tp_tn"]
INFOS=ITEMSETS+CLASSIFICATION_METRICS+DIVERGENCE_METRICS+T_VALUES
pd.set_option('display.max_colwidth', -1)

In [8]:
class_map={"N":0, "P":1}
abbreviations={}

In [9]:
min_sup=0.01
times_de={}
for i in range(0, n_times):
    s=time.time()
    fp_div=FP_DivergenceExplorer(X_one_hot_FP,y_FP,y_predicted, dataset_name=dataset_name)
    FP_fm=fp_div.getFrequentPatternDivergence(min_support=min_sup, class_map=class_map)
    FP_fm=fp_div.t_test_FP(min_sup)
    times_de[i]=time.time()-s
    print(time.time()-s)

3.871400833129883
3.8569350242614746
3.7646257877349854
3.9374942779541016
3.7764055728912354
3.78828501701355
3.933051109313965
3.957368850708008
3.8474748134613037
3.912914276123047


In [37]:
fp_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_fpr.getDivergence()[INFOS].head()

,support,itemsets,tn,tp,fn,fp,fpr,fnr,accuracy,d_fpr,d_fnr,d_accuracy,t_value_fp,t_value_fn,t_value_tp_tn
241,0.12688,"(b=1, c=1, a=1)",0,3187,0,3157,1.0,0.0,0.502364,0.855678,0.0,-0.371516,499.994074,0.441392,57.604258
556,0.12536,"(c=0, b=0, a=0)",0,3119,0,3149,1.0,0.0,0.497607,0.855678,0.0,-0.376273,499.950129,0.452967,58.009797
1236,0.06438,"(b=1, c=1, a=1, h=0)",0,1605,0,1614,1.0,0.0,0.498602,0.855678,0.0,-0.375278,477.324592,0.722607,42.009357
1273,0.06418,"(b=1, c=1, a=1, j=0)",0,1639,0,1570,1.0,0.0,0.510751,0.855678,0.0,-0.363129,475.709473,0.716438,40.597738
1281,0.06418,"(b=1, c=1, a=1, g=0)",0,1586,0,1623,1.0,0.0,0.494235,0.855678,0.0,-0.379645,477.640841,0.726053,42.436330


In [38]:
fp_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_fpr.getDivergence(th_redundancy=0.05)[INFOS].head()

,support,itemsets,tn,tp,fn,fp,fpr,fnr,accuracy,d_fpr,d_fnr,d_accuracy,t_value_fp,t_value_fn,t_value_tp_tn
241,0.12688,"(b=1, c=1, a=1)",0,3187,0,3157,1.000000,0.0,0.502364,0.855678,0.0,-0.371516,499.994074,0.441392,57.604258
556,0.12536,"(c=0, b=0, a=0)",0,3119,0,3149,1.000000,0.0,0.497607,0.855678,0.0,-0.376273,499.950129,0.452967,58.009797
100,0.25018,"(b=1, c=1)",6165,3187,0,3157,0.338661,0.0,0.747622,0.194339,0.0,-0.126258,37.511427,0.441392,30.373557
87,0.25048,"(b=1, a=1)",6180,3187,0,3157,0.338117,0.0,0.747924,0.193795,0.0,-0.125956,37.446486,0.441392,30.327195
29,0.25158,"(c=1, a=1)",6235,3187,0,3157,0.336137,0.0,0.749026,0.191815,0.0,-0.124854,37.209340,0.441392,30.157551


In [10]:
print(mean(times_de.values()))

3.864593744277954


# Slice Finder

## Estimate the log loss

In [11]:
from sklearn.metrics import log_loss
import functools
metric=log_loss
from sklearn.utils.multiclass import unique_labels
classes=unique_labels(y_FP)

y_p = list(map(functools.partial(np.expand_dims, axis=0), y_predict_prob))
y = list(map(functools.partial(np.expand_dims, axis=0), y_FP.values))
loss_list_FP= np.array(list(map(functools.partial(metric, labels=classes), y, y_p)))

In [12]:
import sys
sys.path.append("./SliceFinder_master/")
from SliceFinder_master.slice_finder import SliceFinder

In [13]:
createDir("./log")

### Max workers=1

In [14]:
max_workers=1

n_bin=10
degree=3
k=f"max_w_{max_workers}"
times_sf[k]={}

for i in range(0,n_times):
    s=time.time()
    sf = SliceFinder((X_FP, y_FP), n_bin, loss_list_FP)
    recommendations = sf.find_slice(degree=degree, max_workers=max_workers)
    times_sf[k][i]=time.time()-s
    print(round(time.time()-s,4))

degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
24.8395
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.7712
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.1417
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.094
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.0969
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.7697
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.1044
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.3403
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
17.0197
degree 1
degree : 1, recommended slices : 0
degree 2
deg

In [15]:
print(k, mean(times_sf[k].values()))

max_w_1 18.072201251983643


In [16]:
recom_df=getRecommendationsDf(recommendations)
print(len(recom_df), len(recom_df.drop_duplicates()))

recom_df=recom_df.drop_duplicates()
recom_df.sort_values("effect_size", ascending=False).head(10)

10 8


,itemset,effect_size,metric,size
4,"(c=1, b=1)",1.644455,8.716837,12509
2,"(a=1, b=1)",1.641354,8.706397,12524
5,"(b=0, c=0)",1.636331,8.694748,12509
3,"(b=0, a=0)",1.633236,8.684335,12524
0,"(c=1, a=1)",1.630034,8.668330,12579
1,"(c=0, a=0)",1.621935,8.646364,12579
8,"(h=0, a=1, b=1)",1.403132,8.760897,6363
6,"(c=1, a=1, g=0)",1.393080,8.720665,6428


### Max workers=4

In [17]:
n_bin=10
max_workers=4
degree=3
k=f"max_w_{max_workers}"
times_sf[k]={}
for i in range(0,n_times):
    s=time.time()
    sf = SliceFinder((X_FP, y_FP), n_bin, loss_list_FP)
    recommendations = sf.find_slice(degree=degree, max_workers=max_workers)
    times_sf[k][i]=time.time()-s
    print(round(time.time()-s,4))

degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.6066
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.4587
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.4552
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.9597
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.4987
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.6167
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.4404
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.6548
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 6
degree 3
14.8006
degree 1
degree : 1, recommended slices : 0
degree 2
de

In [18]:
print(k, mean(times_sf[k].values()))

max_w_4 14.597919130325318


In [19]:
recom_df=getRecommendationsDf(recommendations)
print(len(recom_df), len(recom_df.drop_duplicates()))

recom_df=recom_df.drop_duplicates()
recom_df.sort_values("effect_size", ascending=False).head(10)

10 8


,itemset,effect_size,metric,size
4,"(c=1, b=1)",1.644455,8.716837,12509
2,"(a=1, b=1)",1.641354,8.706397,12524
5,"(b=0, c=0)",1.636331,8.694748,12509
3,"(b=0, a=0)",1.633236,8.684335,12524
0,"(c=1, a=1)",1.630034,8.668330,12579
1,"(c=0, a=0)",1.621935,8.646364,12579
8,"(h=0, a=1, b=1)",1.403132,8.760897,6363
6,"(c=1, a=1, g=0)",1.393080,8.720665,6428


### Greater Effect Size Threshold

#### Max workers=1

In [20]:
max_workers=1

n_bin=10
eps=1.65
degree=3
k=f"eps_{eps}_{max_workers}"
times_sf[k]={}

for i in range(0,n_times):
    s=time.time()
    sf = SliceFinder((X_FP, y_FP), n_bin, loss_list_FP)
    recommendations = sf.find_slice(degree=degree, max_workers=max_workers, epsilon=eps)
    times_sf[k][i]=time.time()-s
    print(round(time.time()-s,4))

degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
18.2627
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
17.7712
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
18.1189
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
17.703
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
17.7972
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
18.1838
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
17.6851
degree 1
degree : 1, 

In [21]:
print(k, mean(times_sf[k].values()))

eps_1.65_1 18.228675627708434


In [22]:
recom_df=getRecommendationsDf(recommendations)
print(len(recom_df), len(recom_df.drop_duplicates()))

recom_df=recom_df.drop_duplicates()
recom_df.sort_values("effect_size", ascending=False).head(10)

6 2


,itemset,effect_size,metric,size
3,"(b=0, c=0, a=0)",3.996828,17.352043,6268
0,"(c=1, a=1, b=1)",3.954308,17.187723,6344


#### Max workers=4

In [23]:
max_workers=4

n_bin=10
eps=1.65
degree=3
k=f"eps_{eps}_{max_workers}"
times_sf[k]={}

for i in range(0,n_times):
    s=time.time()
    sf = SliceFinder((X_FP, y_FP), n_bin, loss_list_FP)
    recommendations = sf.find_slice(degree=degree, max_workers=max_workers, epsilon=eps)
    times_sf[k][i]=time.time()-s
    print(round(time.time()-s,4))

degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.1976
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.2871
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.2143
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.7537
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.2436
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.1597
degree 1
degree : 1, recommended slices : 0
degree 2
degree : 2, recommended slices : 0
degree 3
degree : 3, recommended slices : 6
15.1878
degree 1
degree : 1,

In [24]:
print(k, mean(times_sf[k].values()))

eps_1.65_4 15.326845335960389


In [25]:
recom_df=getRecommendationsDf(recommendations)
print(len(recom_df), len(recom_df.drop_duplicates()))

recom_df=recom_df.drop_duplicates()
recom_df.sort_values("effect_size", ascending=False).head(10)

6 2


,itemset,effect_size,metric,size
3,"(b=0, c=0, a=0)",3.996828,17.352043,6268
0,"(c=1, a=1, b=1)",3.954308,17.187723,6344


In [26]:
for k in times_sf:
    print(k, mean(times_sf[k].values())/mean(times_de.values()))

max_w_1 4.676352146649811
max_w_4 3.777348952121936
eps_1.65_1 4.716841363907505
eps_1.65_4 3.9659654675614546


Nearly 4.5 and 3.5 times faster

In [27]:
for k in times_sf:
    print(k, mean(times_sf[k].values()))

max_w_1 18.072201251983643
max_w_4 14.597919130325318
eps_1.65_1 18.228675627708434
eps_1.65_4 15.326845335960389
